# <center>Data Analysis on Hacker News Post</center> 

---


In this project I will be working with a dataset of submissions to popular technology site [Hacker News.](https://news.ycombinator.com/)

![Hacker News Logo.JPG](https://github.com/Clifton-Lee/Exploring-Hacker-News-Post/blob/main/Hacker%20News%20Logo.JPG?raw=true)

Hacker News is a site founded by the startup incubator **Y Combinator**, where user-submitted articles (known as "posts") get votes and comments in a manner similar to Reddit. Hacker News is highly popular in the tech and startup communities, and articles that reach the top of the Hacker News rankings may get hundreds of thousands of visits as a result.

The data set is available [here](https://www.kaggle.com/hacker-news/hacker-news-posts), however please keep in mind that I reduced it from over 300,000 rows to roughly 20,000 rows by eliminating all submissions that did not get any comments and then randomly selecting from the remaining submissions. 

The columns are described below:

- `id`: the unique identifier from Hacker News for the post
- `title`: the title of the post
- `url`: the URL that the posts links to, if the post has a URL
- `num_points`: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: the number of comments on the post
- `author`: the username of the person who submitted the post
- `created_at`: the date and time of the post's submission

I am specifically interested in posts with titles that begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Below are a few examples:

> `Ask HN`: How to improve my personal website?
`Ask HN`: Am I the only one outraged by Twitter shutting down share counts?
`Ask HN`: Aby recent changes to CSS that broke mobile?

Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just something interesting. Below are a few examples:

> `Show HN`: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
`Show HN`: Something pointless I made
`Show HN`: Shanhu.io, a programming playground powered by e8vm

I'll compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?



## Data Understanding & Preparation

I will start by importing the libraries I need and read the dataset into a list of lists. 

**Import Libraries**

In [1]:
import datetime as dt
from csv import reader

**Read In Dataset As List of List** 

In [2]:
# Open the CSV file as a list of list 
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5]) # display the first five rows

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze the data, I need to first remove the row containing the column headers. I will remove that first row next.

In [3]:
# Extract the first row of hn 
headers = hn[0]
hn = hn[1:]

# Display that the extraction was done correctly 
print("The header row is printed below: \n\n",headers, end = '\n\n')
print("The first five rows minus the header row is printed below: \n\n",hn[:5])

The header row is printed below: 

 ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

The first five rows minus the header row is printed below: 

 [['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.co

Now that I've removed the headers from `hn`, I am  ready to filter the data. Since I am  only concerned with post titles beginning with *__Ask HN__* or *__Show HN__*, I'll create new lists of lists containing just the data for those titles.

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    
    if title.lower().startswith('ask hn'):
        ask_posts.append(post)
    elif title.lower().startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print("The number of post with \"Ask HN\": {:,}".format(len(ask_posts)))
print("The number of post with \"Show HN\": {:,}".format(len(show_posts)))
print("The number of post without \"Ask HN\" or \"Show HN\": {:,}".format(len(other_posts)))

The number of post with "Ask HN": 1,744
The number of post with "Show HN": 1,162
The number of post without "Ask HN" or "Show HN": 17,194


## Data Exploration & Analysis

I separated the "ask posts" and the "show posts" into two lists of lists named `ask_posts` and `show_posts`. Below are the first five rows in the `ask_posts` list of lists:

In [5]:
ask_posts[:5]

[['12296411',
  'Ask HN: How to improve my personal website?',
  '',
  '2',
  '6',
  'ahmedbaracat',
  '8/16/2016 9:55'],
 ['10610020',
  'Ask HN: Am I the only one outraged by Twitter shutting down share counts?',
  '',
  '28',
  '29',
  'tkfx',
  '11/22/2015 13:43'],
 ['11610310',
  'Ask HN: Aby recent changes to CSS that broke mobile?',
  '',
  '1',
  '1',
  'polskibus',
  '5/2/2016 10:14'],
 ['12210105',
  'Ask HN: Looking for Employee #3 How do I do it?',
  '',
  '1',
  '3',
  'sph130',
  '8/2/2016 14:20'],
 ['10394168',
  'Ask HN: Someone offered to buy my browser extension from me. What now?',
  '',
  '28',
  '17',
  'roykolak',
  '10/15/2015 16:38']]

Below are the first five rows in the show_posts list of lists:

In [6]:
show_posts[:5]

[['10627194',
  'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform',
  'https://iot.seeed.cc',
  '26',
  '22',
  'kfihihc',
  '11/25/2015 14:03'],
 ['10646440',
  'Show HN: Something pointless I made',
  'http://dn.ht/picklecat/',
  '747',
  '102',
  'dhotson',
  '11/29/2015 22:46'],
 ['11590768',
  'Show HN: Shanhu.io, a programming playground powered by e8vm',
  'https://shanhu.io',
  '1',
  '1',
  'h8liu',
  '4/28/2016 18:05'],
 ['12178806',
  'Show HN: Webscope  Easy way for web developers to communicate with Clients',
  'http://webscopeapp.com',
  '3',
  '3',
  'fastbrick',
  '7/28/2016 7:11'],
 ['10872799',
  'Show HN: GeoScreenshot  Easily test Geo-IP based web pages',
  'https://www.geoscreenshot.com/',
  '1',
  '9',
  'kpsychwave',
  '1/9/2016 20:45']]

Next, I will determine if ask posts or show posts receive more comments on average.

In [7]:
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments/len(ask_posts)

print('The average number of comments label "Ask HN" is: {:.2f}'.format(avg_ask_comments))

The average number of comments label "Ask HN" is: 14.04


In [8]:
total_show_comments = 0

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments/len(show_posts)

print('The average number of comments label "Show HN" is: {:.2f}'.format(avg_show_comments))

The average number of comments label "Show HN" is: 10.32


Based on the information I've gathered so far, I've gleaned that "ask posts" get on average more comments than "show posts". Since "ask posts" are more likely to get comments than other types of postings, I will concentrate my remaining analysis only on these types of posts.

Next, I'll determine if "ask posts" created at a certain time are more likely to attract comments. I'll use the following steps to perform this analysis:

1. Calculate the number of "ask posts" created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments "ask posts" receive by hour created.

In [9]:
result_list = []

for post in ask_posts:
    result_list.append([post[6],int(post[4])]) # created_at column is the 7th column in ask_posts hence index 6 
    
counts_by_hour = {}
comments_by_hour = {}

for item in result_list:
    the_date = item[0]
    the_date = dt.datetime.strptime(the_date,'%m/%d/%Y %H:%M')
    the_hour = dt.datetime.strftime(the_date,'%H')
    
    if the_hour not in counts_by_hour:
        counts_by_hour[the_hour] = 1
        comments_by_hour[the_hour] = item[1]
    else:
        counts_by_hour[the_hour] += 1
        comments_by_hour[the_hour] += item[1]

In [10]:
print("The count of post by hour is displyed below: \n",counts_by_hour,end = '\n\n')
print("The comments by hour is displayed below: \n",comments_by_hour)

The count of post by hour is displyed below: 
 {'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}

The comments by hour is displayed below: 
 {'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


In the previous code cell, I created two dictionaries:

- `counts_by_hour`: contains the number of ask posts created during each hour of the day.
- `comments_by_hour`: contains the corresponding number of comments ask posts created at each hour received.

Next, I'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [11]:
avg_by_hour = []

for hour in comments_by_hour:
    avg_comment = round(comments_by_hour[hour] / counts_by_hour[hour],0)
    avg_by_hour.append([hour,avg_comment])
    
avg_by_hour

[['09', 6.0],
 ['13', 15.0],
 ['10', 13.0],
 ['14', 13.0],
 ['16', 17.0],
 ['23', 8.0],
 ['12', 9.0],
 ['17', 11.0],
 ['15', 39.0],
 ['21', 16.0],
 ['20', 22.0],
 ['02', 24.0],
 ['18', 13.0],
 ['03', 8.0],
 ['05', 10.0],
 ['19', 11.0],
 ['01', 11.0],
 ['22', 7.0],
 ['08', 10.0],
 ['04', 7.0],
 ['00', 8.0],
 ['06', 9.0],
 ['07', 8.0],
 ['11', 11.0]]

Although I now have the results I need, this format makes it difficult to identify the hours with the highest values. I'll finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [12]:
swap_avg_by_hour = []

for item in avg_by_hour:
    swap_avg_by_hour.append([item[1],item[0]])
    
print(swap_avg_by_hour)

[[6.0, '09'], [15.0, '13'], [13.0, '10'], [13.0, '14'], [17.0, '16'], [8.0, '23'], [9.0, '12'], [11.0, '17'], [39.0, '15'], [16.0, '21'], [22.0, '20'], [24.0, '02'], [13.0, '18'], [8.0, '03'], [10.0, '05'], [11.0, '19'], [11.0, '01'], [7.0, '22'], [10.0, '08'], [7.0, '04'], [8.0, '00'], [9.0, '06'], [8.0, '07'], [11.0, '11']]


In [13]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print("Top 5 Hours for Ask Post Comments:", end = '\n\n')

for avg, hour in sorted_swap[:5]:
    given_hour = dt.datetime.strptime(hour,"%H")
    given_hour = dt.datetime.strftime(given_hour,"%H:%M")
    
    print("{} : {:.2f} average comments per post".format(given_hour, avg))

Top 5 Hours for Ask Post Comments:

15:00 : 39.00 average comments per post
02:00 : 24.00 average comments per post
20:00 : 22.00 average comments per post
16:00 : 17.00 average comments per post
21:00 : 16.00 average comments per post


## Conclusion

According to the data gathered, the three best hours to make a post in order to maximize the probability of receiving comments are as follows:

1. 3 p.m. Eastern Time in the United States

2. 2 a.m. Eastern Time in the United States

3. 8 p.m. Eastern Time in the United States
